### Create regional panel of schools from School API
This code creates the list of schools at town level based on School API provided by Korea Elementary School Principal's Association (KESPA) <br>
Description on School API is here : https://www.schoolinfo.go.kr/ng/go/pnnggo_a01_m0.do <br>
Last modified: May 21 2021 by Imryoung Jeong(neptune0118@gmail.com)

In [1]:
import requests, certifi, json
import pandas as pd
import geopandas as gpd
import numpy as np
import googlemaps

from pandas.io.json import json_normalize
from functools import reduce

###### Get the full list of existing schools from School API
---

In [ ]:
dir_out = "/Users/imryoung/Dropbox/Korea_Protestantism/Data/rawdata/School/"

In [ ]:
APIKEY = "INSERT_YOUR_API_KEY"
API_TYPE = "0"
YEAR = "2019"
SCHL_CATEGORY = "04" # Tech high

In [10]:
jsonUrl = "https://www.schoolinfo.go.kr/openApi.do?apiKey={}&apiType={}&pbanYr={}&schulKndCode={}".format(APIKEY,API_TYPE,YEAR,SCHL_CATEGORY)

json = requests.get(jsonUrl, verify = False)
df = pd.json_normalize(json.json()['list'])

df['FOND_YMD'] = pd.to_datetime(df['FOND_YMD'])

df2 = df[['FOND_SC_CODE','LTTUD','LGTUD','COEDU_SC_CODE','ATPT_OFCDC_ORG_NM',
              'SCHUL_CODE','ZIP_CODE','SCHUL_NM']].copy()

df2['FOND_YEAR'] = df['FOND_YMD'].dt.year
df2['TECH_SCHUL'] = (df['HS_KND_SC_NM'] == '특성화고등학교').astype(int)

df2.to_csv(dir_out+"Existing_school_tech_high.csv")

/Users/imryoung/opt/anaconda3/envs/geo-env/lib/python3.9/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.schoolinfo.go.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


###### Extract the coordinates of closed schools from Google Maps API
----

In [151]:
closed = pd.read_csv(r2 + '폐교정보-서울.csv')

df = pd.DataFrame()

for i in range(len(closed)) :
    
    address = closed['주소'][i]
    
    geocode_result = gmaps.geocode((address), language = 'ko')
    coordinate = pd.json_normalize(geocode_result[0]['geometry']['location'])
    
    df = df.append(coordinate, ignore_index = True)

df['SCHUL_NAME'] = closed['학교명']
df['CLOSED_YEAR'] = closed['폐교연도']
df['ADDR'] = closed['주소']

df.to_csv(dir_out + 'closed_school.csv')

In [21]:
# After some manual cleaning...
df = pd.read_csv(dir_out + "Closed_school_V3.csv")
df.head(3)

,Unnamed: 0,lat,lng,SCHUL_NAME,SCHUL_LEVEL,OPEN_YEAR,CLOSED_YEAR,ADDR,TECH_SCHUL
0,0,37.566476,126.851037,공진중,2,1993.0,2020,서울특별시 강서구 가양동 1476 공진중학교,0
1,1,37.561165,126.864010,염강초,1,1994.0,2020,서울특별시 강서구 가양동 1492 염강초등학교,0
2,2,35.132116,129.043763,(구)경남여자중학교,2,1927.0,2009,부산광역시 동구 수정5동 563-1,0


###### Count the number of Schools
----

In [ ]:
dir_shp = "/Users/imryoung/Dropbox/Korea_Protestantism/GIS/Koreagov_1918boundary/"

shp1918 = gpd.read_file(dir_shp + "province_shp/myeon.shp",
                       encoding = 'euc-kr',
                       ignore_fields = ['설치일자','폐지일자','비고','사용여부','생성일시','Area']).set_crs(epsg = 4326)

In [7]:
# Elementary Schools : 1 , Junior high : 2 , High : 3
dir_out = "/Users/imryoung/Dropbox/Korea_Protestantism/Data/rawdata/School/"

schl = pd.read_csv(dir_out + "Existing_school_tech_high.csv", encoding = 'euc-kr')
closed = pd.read_csv(dir_out + "Closed_school_V3.csv")

closed['OPEN_YEAR'].isnull().sum()
closed.dropna(subset=['OPEN_YEAR'], inplace=True)
closed = closed.loc[closed['SCHUL_LEVEL'] == 3]

gdf_schl = gpd.GeoDataFrame(schl[['COEDU_SC_CODE','SCHUL_CODE','SCHUL_NM','FOND_YEAR']], 
                           geometry = gpd.points_from_xy(schl.LGTUD, schl.LTTUD)).set_crs(epsg=4326)

gdf_closed = gpd.GeoDataFrame(closed[['SCHUL_NAME','OPEN_YEAR','CLOSED_YEAR']],
                             geometry = gpd.points_from_xy(closed.lng, closed.lat)).set_crs(epsg=4326)

In [8]:
gdf_schl

,COEDU_SC_CODE,SCHUL_CODE,SCHUL_NM,FOND_YEAR,geometry
0,남녀공학,S000003511,서울교육대학교부설초등학교,1953,POINT (127.01542 37.49074)
1,남녀공학,S000003563,서울대학교사범대학부설초등학교,1946,POINT (127.00309 37.57702)
2,남녀공학,S010000738,서울개일초등학교,1987,POINT (127.05774 37.48621)
3,남녀공학,S010000741,서울구룡초등학교,1983,POINT (127.05175 37.48119)
4,남녀공학,S010000742,서울논현초등학교,1972,POINT (127.00000 37.50000)
...,...,...,...,...,...
6274,남녀공학,S170000801,월랑초등학교,2007,POINT (126.47795 33.49283)
6275,남녀공학,S170000834,이도초등학교,2012,POINT (126.53199 33.48812)
6276,남녀공학,S170000836,도련초등학교,2015,POINT (126.58430 33.51402)
6277,남녀공학,S170000837,삼화초등학교,2014,POINT (126.57701 33.51522)


In [9]:
gdf_schl = gpd.sjoin(gdf_schl, shp1918, how = 'left', op = 'within')
gdf_closed = gpd.sjoin(gdf_closed, shp1918, how = 'left', op = 'within')

In [26]:
print(gdf_schl['지역코드'].isnull().sum())
print(gdf_closed['지역코드'].isnull().sum())

80
1


In [10]:
def calculator(exist, closed, year) :
    
    varname = 'School_TechHigh_' + str(year)
    
    df1 = exist.groupby('지역코드').apply(lambda x : (x['FOND_YEAR'] < year).sum()).reset_index(name = 'exist')
    df2 = closed.groupby('지역코드').apply(lambda x : ((x['OPEN_YEAR'] < year) 
                                                    & (x['CLOSED_YEAR'] > year)).sum()).to_dict()
    df1['closed'] = df1['지역코드'].map(df2)
    df1['closed'].fillna(0, inplace = True)
    
    result = df1[['지역코드']].copy()
    result[varname] = df1['exist'] + df1['closed'].astype(int)
    
    return result

In [12]:
result = calculator(gdf_schl, gdf_closed, 1934)

result.to_csv("/Users/imryoung/work/project_Protestantantism/data/school/school_elem_1934.csv",
             index=False)

In [33]:
full_list = []

for year in range(1960,2020,5) :
    
    full_list.append(calculator(gdf_schl, gdf_closed, year))
    
final = reduce(lambda x,y : pd.merge(x,y, on='지역코드', how='outer'), full_list)


In [34]:
final

,지역코드,School_TechHigh_1960,School_TechHigh_1965,School_TechHigh_1970,School_TechHigh_1975,School_TechHigh_1980,School_TechHigh_1985,School_TechHigh_1990,School_TechHigh_1995,School_TechHigh_2000,School_TechHigh_2005,School_TechHigh_2010,School_TechHigh_2015
0,A01_01,0,0,0,0,0,0,0,0,0,0,0,0
1,A01_05,0,0,0,0,0,0,0,0,0,0,0,0
2,A01_06,0,0,0,0,0,0,0,0,0,0,0,0
3,A02_01,0,0,0,1,1,1,1,1,1,1,1,1
4,A02_02,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
713,H21_07,0,0,0,0,0,0,0,0,0,0,0,0
714,K10_01,0,0,0,0,1,1,1,1,1,1,1,1
715,M13_05,0,0,0,0,0,0,0,0,0,0,0,0
716,M16_13,0,0,0,0,0,0,0,0,0,0,0,0


In [35]:
final.to_csv(dir_out + "School_TechHigh_60_15.csv")